In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Use geopy library to find coordinates of Los Mochis

In [2]:
address = 'Los Mochis, Sinaloa'

geolocator = Nominatim(user_agent="lm_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Los Mochis are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Los Mochis are 25.7928058, -108.990188.


## Define foursquare credentials

In [26]:
# @hidden_cell

CLIENT_ID = 'AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR' # your Foursquare ID
CLIENT_SECRET = '5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR
CLIENT_SECRET:5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK


In [5]:
#We'll use four locations in Los Mochis to extract more than 100 results, since each venue only gives 100 results
long1=-108.942
lat1=25.8005
long2=-109.048
lat2=25.7911
long3=-108.988
lat3=25.8247
long4=-108.986
lat4=25.7544

### Get venues in Los Mochis
#### Table one

In [6]:
LIMIT=100
radius=5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat1, 
    long1, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR&client_secret=5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK&v=20180605&ll=25.8005,-108.942&radius=5000&limit=100'

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ccf5873dd57972414e5f04b'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ff8b7f7e4b0e8d00e3701b9-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ce941735',
         'name': 'Seafood Restaurant',
         'pluralName': 'Seafood Restaurants',
         'primary': True,
         'shortName': 'Seafood'}],
       'id': '4ff8b7f7e4b0e8d00e3701b9',
       'location': {'cc': 'MX',
        'country': 'México',
        'distance': 2635,
        'formattedAddress': ['México'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 25.80349096940428,
          'lng': -108.96809113698511}],
        'lat': 25.80349096940428,
        'lng': -108.96809113698511}

### Create a function to extract venue category

In [8]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Put that into a DataFrame

In [9]:
venues = results['response']['groups'][0]['items']
    
lm_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
lm_venues =lm_venues.loc[:, filtered_columns]

# filter the category for each row
lm_venues['venue.categories'] = lm_venues.apply(get_category_type, axis=1)

# clean columns
lm_venues.columns = [col.split(".")[-1] for col in lm_venues.columns]

lm_venues.head()

,name,categories,lat,lng
0,Mariscos la memoria,Seafood Restaurant,25.803491,-108.968091
1,Ciclopista Cerro de La Memoria,Athletics & Sports,25.810077,-108.976175
2,Cerro de la Memoria (Mirador),Scenic Lookout,25.808307,-108.973012
3,Farmacia Guadalajara Las Fuentes,Pharmacy,25.803706,-108.979477
4,Tacos Punta Arena,Taco Place,25.791606,-108.980147


### Create table 2

In [11]:
LIMIT=100
radius=5000
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat2, 
    long2, 
    radius, 
    LIMIT)
url2 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR&client_secret=5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK&v=20180605&ll=25.7911,-109.048&radius=5000&limit=100'

In [12]:
results2 = requests.get(url2).json()


In [13]:
venues2 = results2['response']['groups'][0]['items']
    
lm_venues2 = json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
lm_venues2 =lm_venues2.loc[:, filtered_columns]

# filter the category for each row
lm_venues2['venue.categories'] = lm_venues2.apply(get_category_type, axis=1)

# clean columns
lm_venues2.columns = [col.split(".")[-1] for col in lm_venues2.columns]

lm_venues2.head()

,name,categories,lat,lng
0,Estadio Centenario,Soccer Stadium,25.797691,-109.031148
1,birrieria mexico,Taco Place,25.806653,-109.022864
2,J&R Ribs LA COSTILLERIA,Bistro,25.792341,-109.010940
3,Country Club,Golf Course,25.792104,-109.009421
4,Kinatia,Tea Room,25.793400,-109.010284


### Create table 3

In [14]:
LIMIT=100
radius=5000
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat2, 
    long2, 
    radius, 
    LIMIT)
url3 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR&client_secret=5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK&v=20180605&ll=25.7911,-109.048&radius=5000&limit=100'

In [15]:
results3 = requests.get(url3).json()

In [16]:
venues3 = results3['response']['groups'][0]['items']
    
lm_venues3 = json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
lm_venues3 =lm_venues3.loc[:, filtered_columns]

# filter the category for each row
lm_venues3['venue.categories'] = lm_venues3.apply(get_category_type, axis=1)

# clean columns
lm_venues3.columns = [col.split(".")[-1] for col in lm_venues3.columns]

lm_venues3.head()

,name,categories,lat,lng
0,Estadio Centenario,Soccer Stadium,25.797691,-109.031148
1,birrieria mexico,Taco Place,25.806653,-109.022864
2,J&R Ribs LA COSTILLERIA,Bistro,25.792341,-109.010940
3,Country Club,Golf Course,25.792104,-109.009421
4,Kinatia,Tea Room,25.793400,-109.010284


### Create table 4

In [21]:
LIMIT=100
radius=5000
url4 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat2, 
    long2, 
    radius, 
    LIMIT)
url4 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR&client_secret=5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK&v=20180605&ll=25.7911,-109.048&radius=5000&limit=100'

In [22]:
results4 = requests.get(url4).json()

In [23]:
venues4 = results4['response']['groups'][0]['items']
    
lm_venues4 = json_normalize(venues4) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
lm_venues4 =lm_venues4.loc[:, filtered_columns]

# filter the category for each row
lm_venues4['venue.categories'] = lm_venues4.apply(get_category_type, axis=1)

# clean columns
lm_venues4.columns = [col.split(".")[-1] for col in lm_venues4.columns]

lm_venues4.head()

,name,categories,lat,lng
0,Estadio Centenario,Soccer Stadium,25.797691,-109.031148
1,birrieria mexico,Taco Place,25.806653,-109.022864
2,J&R Ribs LA COSTILLERIA,Bistro,25.792341,-109.010940
3,Country Club,Golf Course,25.792104,-109.009421
4,Kinatia,Tea Room,25.793400,-109.010284


### Stack four tables 

In [24]:
lm_allvenues = pd.concat([lm_venues, lm_venues2,lm_venues3,lm_venues4])
lm_allvenues.index=range(len(lm_allvenues))
lm_allvenues.head()

,name,categories,lat,lng
0,Mariscos la memoria,Seafood Restaurant,25.803491,-108.968091
1,Ciclopista Cerro de La Memoria,Athletics & Sports,25.810077,-108.976175
2,Cerro de la Memoria (Mirador),Scenic Lookout,25.808307,-108.973012
3,Farmacia Guadalajara Las Fuentes,Pharmacy,25.803706,-108.979477
4,Tacos Punta Arena,Taco Place,25.791606,-108.980147
